In [1]:
result_files = "../trialgpt_results[67]/matching_results_sigir_gpt-4-turbo.json"
qrel = "../sigir/extracted_titles_and_numbers.csv"

In [2]:
import json
import pandas as pd
df = pd.read_csv(qrel)

sigir_files = "./sigir/clinicaltrials.gov-16_dec_2015"
with open(result_files, "r")as f:
    results = json.load(f)
patient_note = results.keys()
patient_topic = []

for i in patient_note:  
    i = i.split("-")[1]
    patient_topic.append(df.loc[df["NUM"]==int(i)].TITLE.values[0])
    

In [3]:
patient_topic[1]

'An 8-year-old male presents in March to the ER with fever up to 39 C, dyspnea and cough for 2 days. He has just returned from a 5 day vacation in Colorado. Parents report that prior to the onset of fever and cough, he had loose stools. He denies upper respiratory tract symptoms. On examination he is in respiratory distress and has bronchial respiratory sounds on the left. A chest x-ray shows bilateral lung infiltrates.\n'

## Detected the sensitive information 

In [ ]:
from transformers import AutoModel, AutoTokenizer
sensitive_words = "sensitive_list/"
model_name = "gpt2"

def filter_senstive():
    
def replace_senstive():
    
def 


## Replace the sensitive information 

In [13]:
from transformers import pipeline

# 加载预训练的命名实体识别模型
ner_model = pipeline("ner", model="dbmdz/bert-large-cased-finetuned-conll03-english")

# 需要分析的文本
text = '''An 8-year-old male presents in March to the ER with fever up to 39 C, dyspnea and cough for 2 days. He has just returned from a 5 day vacation in Colorado. Parents report that prior to the onset of fever and cough, he had loose stools. He denies upper respiratory tract symptoms. On examination he is in respiratory distress and has bronchial respiratory sounds on the left. A chest x-ray shows bilateral lung infiltrates.'''

# 进行命名实体识别
entities = ner_model(text)

# 打印结果
for entity in entities:
    print(f"Entity: {entity['word']}, Label: {entity['entity']}")


/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

KeyboardInterrupt



## Fairness detection

## Calibrating the distribution of counterfactual